### Inputs and Imports

In [2]:
from collections import Counter

day = '06'
        
with open(f'Inputs\day_{day}.txt', 'r') as input_file: 
    # convert input into list of ints
    input_file_list = [int(x) for x in input_file.read().split(",")]       
with open(f'Inputs\day_{day}_sample.txt', 'r') as input_file: 
    # convert input into list of ints
    sample_input_file_list = [int(x) for x in input_file.read().split(",")]        
      
# sample_input_file_list    
########################        
# Part One Sample Answer: 5934 (80 days) / 26 (18 days)
########################
# Part Two Sample Answer: 26984457539 (80 days) 
########################

### Part One

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

- After one day, its internal timer would become 2.
- After another day, its internal timer would become 1.
- After another day, its internal timer would become 0.
- After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
- After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.

A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

3,4,3,1,2
This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:
```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
```
Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

### Part Two

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of 26984457539 lanternfish!

In [6]:
# lfish_ages = sample_input_file_list.copy()
lfish_ages = input_file_list.copy()

# create a dictionary of possible fish ages (0 - 8) with a count of how many times they occur in the lfish_ages input
def create_inital_state(lfish_ages):
    # create a dict for each possible age (0-8)
    ages_dict = dict.fromkeys(list(range(9)), 0)

    # get the initial state, a count of each age in a dict
    initial_state = dict(Counter(lfish_ages))

    # update the ages_dict with the number of each age the initial state
    for key,val in initial_state.items():
        ages_dict[key] = val
    return ages_dict

# simulates one day in the lifecylce of the lantern fish and outputs and updated ages dictionary
def sim_one_day(ages_dict):    
    # create a copy of the ages dictionary
    new_ages_dict = ages_dict.copy()
    # iterate through the ages dict and update the values in the copy made above
    for key, val in ages_dict.items():
        # if day zero then 'reset' these fish to age 6 but also add new fish at age 8
        if key == 0:
            new_ages_dict[8] += val
            new_ages_dict[6] += val
            new_ages_dict[0] -= val
        # otherwise just remove the amount of fish from the current 'age' and add to the total for the next lowest age
        else:
            new_ages_dict[key]   -= val
            new_ages_dict[key-1] += val
    return new_ages_dict

# sim through the lifecycle for x amount of days (one of the params) 
# print the total number of lanternfish in the end state
def sim_multiple_days(ages_dict, days_to_sim):
    days = 0
    # sim one day and add to the total days until we reach the days_to_sim param
    while days < days_to_sim:
        ages_dict = sim_one_day(ages_dict)
        days += 1
    
    # sum all the values for each lanternfish 'age' 
    total_fish = 0
    for key, val in ages_dict.items():
        total_fish += val
    
    print(f'After {days_to_sim} days there will be {total_fish} lanternfish in total')
    return ages_dict

ages_dict = create_inital_state(lfish_ages)

# Part One
final_ages_dict = sim_multiple_days(ages_dict, 80)
# Part Two
final_ages_dict = sim_multiple_days(ages_dict, 256)

final_ages_dict = sim_multiple_days(ages_dict, 1000000)

After 80 days there will be 350605 lanternfish in total
After 256 days there will be 1592778185024 lanternfish in total
After 1000000 days there will be ... lanternfish in total


### Failed Methods

Both the below methods failed, they were each a form of iterating through a list and then creating a new list in the same format as the input, this got exponentially too big. Depending on whether using the sample or the actual puzzle input would just get too big to iterate over before getting to day 256. (at around day 180 it was around 100 million elements long and was taking forever to iterate through. 

In [4]:
lfish_ages = sample_input_file_list.copy()
# lfish_ages = input_file_list.copy()

def fish_lifecycle_sim(lfish_ages, days_to_sim):
    print(f'day 0\n{lfish_ages}\n')
    counter = 1
    while counter < days_to_sim+1:
#         print(f'day {counter}\n{lfish_ages}')
        print(f'day {counter} / length = {len(lfish_ages)}')
        for index, lfish in enumerate(lfish_ages):
            age = lfish_ages[index]
            if age == 0:
                lfish_ages[index] = 6
                lfish_ages.append(9) # adding a 9 not an 8 as the for loop will hit this number and minus 1
            else:
                lfish_ages[index] = lfish_ages[index]-1
#         print(f'{lfish_ages}\n')
        counter += 1
    print(f'there are now {len(lfish_ages)} lanternfish')    

# fish_lifecycle_sim(lfish_ages, 80)
fish_lifecycle_sim(lfish_ages, 256)

day 0
[3, 4, 3, 1, 2]

day 1 / length = 5
day 2 / length = 5
day 3 / length = 6
day 4 / length = 7
day 5 / length = 9
day 6 / length = 10
day 7 / length = 10
day 8 / length = 10
day 9 / length = 10
day 10 / length = 11
day 11 / length = 12
day 12 / length = 15
day 13 / length = 17
day 14 / length = 19
day 15 / length = 20
day 16 / length = 20
day 17 / length = 21
day 18 / length = 22
day 19 / length = 26
day 20 / length = 29
day 21 / length = 34
day 22 / length = 37
day 23 / length = 39
day 24 / length = 41
day 25 / length = 42
day 26 / length = 47
day 27 / length = 51
day 28 / length = 60
day 29 / length = 66
day 30 / length = 73
day 31 / length = 78
day 32 / length = 81
day 33 / length = 88
day 34 / length = 93
day 35 / length = 107
day 36 / length = 117
day 37 / length = 133
day 38 / length = 144
day 39 / length = 154
day 40 / length = 166
day 41 / length = 174
day 42 / length = 195
day 43 / length = 210
day 44 / length = 240
day 45 / length = 261
day 46 / length = 287
day 47 / leng

KeyboardInterrupt: 

In [93]:
lfish_ages = sample_input_file_list.copy()
# lfish_ages = input_file_list.copy()

def fish_lifecycle_one_day_sim(lfish_ages):
    next_day = []
    new_fish = 0
    for i in lfish_ages:
        if i > 0:
            next_day.append(i-1)
        else:
            next_day.append(6)
            new_fish += 1 # adding a new fish to the new_fish counter

    if new_fish != 0:
        for _ in range(new_fish): next_day.append(8) # adding a new fish starting at day 8 for each new fish produced
    return next_day

def increment_lifecycle(lfish_ages, days_to_increment):
    print(f'Initial State: {lfish_ages}')
    day = 0
    while day < days_to_increment:
        lfish_ages = fish_lifecycle_one_day_sim(lfish_ages)
        day += 1
        print(f'after day {day}: {len(lfish_ages)}')
    return lfish_ages

increment_lifecycle(lfish_ages, 80) 
increment_lifecycle(lfish_ages, 256) 